<a href="https://colab.research.google.com/github/paul-kim88/pythonProject2/blob/master/scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lxml[html_clean] readability-lxml pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 49.7 MB/s eta 0:00:00


In [2]:
import requests
from bs4 import BeautifulSoup
from readability import Document
import pandas as pd
import re, html, time
from io import BytesIO
from pdfminer.high_level import extract_text

def duckduckgo_scrape(query, time='d'):
    url = f"https://duckduckgo.com/html/?q={query}&t=h_&df={time}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        results = []
        for result in soup.find_all("div", class_="result"):
            title_element = result.find("a", class_="result__a")
            if title_element:
                title = title_element.text
                link = title_element["href"]
                results.append({"title": title, "link": link})
        return results
    except Exception as e:
        print(f"Error during request: {e}")
        return None

def extract(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    text = soup.get_text(separator="\n", strip=True)
    return html.unescape(text)

def process_link(result, session):
    try:
        response = session.get(result['link'], timeout=10)
        response.raise_for_status()

        if result['link'].endswith('.pdf') or 'application/pdf' in response.headers.get('Content-Type', ''):
            pdf_file = BytesIO(response.content)
            content = extract_text(pdf_file)
        else:
            doc = Document(response.content)
            content = extract(doc.summary())
        if len(content) < 100:
            return None
        return {"title": result['title'], "link": result['link'], "content": content}

    except Exception as e:
        print(f"Error processing {result['link']}: {e}")
        return None

if __name__ == "__main__":
    search_query = input("Type What you want : ")
    start_time = time.time()
    search_results = duckduckgo_scrape(search_query, '3d')
    results = []

    if search_results:
        with requests.Session() as session:
            for result in search_results:
                if 'wiki' in result['link']:
                    continue
                # print(f"Title: {result['title']}")
                # print(f"Link: {result['link']}")
                # print("-" * 20)
                processed = process_link(result, session)
                if processed:
                    results.append(processed)

    search_df = pd.DataFrame(results)
    end_time = time.time()
    print("실행 시간:", end_time - start_time, "초")


Type What you want : LG화학 코스모프로프
실행 시간: 16.740375995635986 초


In [3]:
search_df

,title,link,content
0,"Lg화학, 세계 최대 뷰티 박람회서 '이산화탄소 플라스틱' 첫선 | 연합뉴스",https://www.yna.co.kr/view/AKR20240320022900003,(서울=연합뉴스) 장하나 기자 = LG화학[051910]이 오는 21일 이탈리아 볼...
1,친환경 플라스틱 속도내는 Lg화학…Co₂플라스틱 첫선,https://www.mk.co.kr/news/business/10969384,PEC 적용한 화장품 용기 전시\n현존 CO₂플라스틱 중 생산성 최고\n생분해·재활...
2,"Lg화학, 세계 최대 뷰티 박람회서 '이산화탄소 플라스틱' 첫선",https://www.hankyung.com/article/202403202491Y,LG화학이 오는 21일 이탈리아 볼로냐에서 열리는 세계 최대 뷰티 박람회 '코스모프...
3,Lg화학 이산화탄소 플라스틱 코스모프로프 볼로냐 2024에서 첫선 : 네이버 블로그,https://m.blog.naver.com/fyndignews/223389616071,LG화학이 이산화탄소 플라스틱 시장 개척에 나선다.\n​\nLG화학은 21일 이탈리...
4,"﻿Lg화학, '코스모프로프 볼로냐 2024'에서 Co2 플라스틱 전시",https://www.lcnews.co.kr/news/articleView.html...,LG화학의 이산화탄소 플라스틱 PEC [사진=LG화학 제공]\n라이센스뉴스 = 김진...
5,"Lg화학, 이산화탄소 플라스틱 적용한 화장품 용기 첫 선",http://businessnews.chosun.com/site/data/html_...,LG화학의 이산화탄소 플라스틱 PEC./LG화학 제공\nLG화학이 이산화탄소 플라스...
6,"Lg화학, 코스모프로프 전시회서 혁신소재 용기 첫 선",https://www.inews24.com/view/1699093,[아이뉴스24 이시은 기자] LG화학은 이탈리아 볼로냐에서 개최하는 '코스모프로프 ...
7,"Lg화학, '코스모프로프 볼로냐 2024'서 Co2플라스틱 첫 선",https://www.insightkorea.co.kr/news/articleVie...,LG화학이 이산화탄소 플라스틱 시장 개척에 나선다. <LG화학>\n[인사이트코리아=...
8,"Lg화학, 세계 최대 뷰티 박람회서 친환경 화장품 용기 첫 선 < 철강/화학 < I...",https://www.thepublic.kr/news/articleView.html...,[더퍼블릭=이유정 기자] LG화학은 오는 21일 이탈리아 볼로냐에서 개최하는 ‘코스...


In [4]:
import requests
from bs4 import BeautifulSoup
from readability import Document
import pandas as pd
import re, html
from io import BytesIO
from pdfminer.high_level import extract_text
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

def duckduckgo_scrape(query, time='d'):
    url = f"https://duckduckgo.com/html/?q={query}&t=h_&df={time}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        results = []
        for result in soup.find_all("div", class_="result"):
            title_element = result.find("a", class_="result__a")
            if title_element:
                title = title_element.text
                link = title_element["href"]
                results.append({"title": title, "link": link})
        return results
    except Exception as e:
        print(f"Error during request: {e}")
        return None

def extract(html_text):
    soup = BeautifulSoup(html_text, 'html.parser')
    text = soup.get_text(separator="\n", strip=True)
    return html.unescape(text)

def process_link(result, session):
    try:
        response = session.get(result['link'], timeout=10)
        response.raise_for_status()

        if result['link'].endswith('.pdf') or 'application/pdf' in response.headers.get('Content-Type', ''):
            pdf_file = BytesIO(response.content)
            content = extract_text(pdf_file)
        else:
            doc = Document(response.content)
            content = extract(doc.summary())
        if len(content) < 100:
            return None  # 내용이 100자 미만이면 건너뜁니다.
        return {"title": result['title'], "link": result['link'], "content": content}
    except Exception as e:
        print(f"Error processing {result['link']}: {e}")
        return None

if __name__ == "__main__":
    search_query = input("Type What you want : ")
    start_time = time.time()
    search_results = duckduckgo_scrape(search_query, '3d')
    results = []

    if search_results:
        with requests.Session() as session:
          with ThreadPoolExecutor(max_workers=20) as executor:
              future_to_result = {}
              for result in search_results:
                  if 'wiki' in result['link']:
                      continue
                  # print(f"Title: {result['title']}")
                  # print(f"Link: {result['link']}")
                  # print("-" * 20)

                  future = executor.submit(process_link, result, session)
                  future_to_result[future] = result

              for future in as_completed(future_to_result):
                  processed = future.result()
                  if processed:
                      results.append(processed)

    search_df = pd.DataFrame(results)
    end_time = time.time()
    print("실행 시간:", end_time - start_time, "초")


Type What you want : LG화학 코스모프로프
실행 시간: 2.675489664077759 초


In [5]:
search_df

,title,link,content
0,Lg화학 이산화탄소 플라스틱 코스모프로프 볼로냐 2024에서 첫선 : 네이버 블로그,https://m.blog.naver.com/fyndignews/223389616071,LG화학이 이산화탄소 플라스틱 시장 개척에 나선다.\n​\nLG화학은 21일 이탈리...
1,"Lg화학, 세계 최대 뷰티 박람회서 친환경 화장품 용기 첫 선 < 철강/화학 < I...",https://www.thepublic.kr/news/articleView.html...,[더퍼블릭=이유정 기자] LG화학은 오는 21일 이탈리아 볼로냐에서 개최하는 ‘코스...
2,"Lg화학, '코스모프로프 볼로냐 2024'서 Co2플라스틱 첫 선",https://www.insightkorea.co.kr/news/articleVie...,LG화학이 이산화탄소 플라스틱 시장 개척에 나선다. <LG화학>\n[인사이트코리아=...
3,친환경 플라스틱 속도내는 Lg화학…Co₂플라스틱 첫선,https://www.mk.co.kr/news/business/10969384,PEC 적용한 화장품 용기 전시\n현존 CO₂플라스틱 중 생산성 최고\n생분해·재활...
4,"Lg화학, '코스모프로프 볼로냐2024'서 친환경 플라스틱 소재 전시",https://www.100ssd.co.kr/news/articleView.html...,이산화탄소로 만든 차세대 혁신소재 PEC 적용한 화장품 용기 선봬\nLG화학의 연구...
5,"﻿Lg화학, '코스모프로프 볼로냐 2024'에서 Co2 플라스틱 전시",https://www.lcnews.co.kr/news/articleView.html...,LG화학의 이산화탄소 플라스틱 PEC [사진=LG화학 제공]\n라이센스뉴스 = 김진...
6,"Lg화학, 세계 최대 뷰티 박람회서 '이산화탄소 플라스틱' 첫선 | 연합뉴스",https://www.yna.co.kr/view/AKR20240320022900003,(서울=연합뉴스) 장하나 기자 = LG화학[051910]이 오는 21일 이탈리아 볼...
7,"Lg화학, 세계 최대 뷰티 박람회 '코스모프로프 볼로냐'에서 친환경 Co2 플라스틱 전시",https://www.dailybizon.com/news/articleView.ht...,LG화학의 이산화탄소 플라스틱 PEC(Poly Ethylene Carbonate)\...
8,"LG화학, 친환경 화장품 용기로 뷰티 시장 공략 - ZDNet korea",https://zdnet.co.kr/view/?no=20240320095448,LG화학이 이산화탄소 플라스틱 시장 개척에 나선다.\nLG화학은 21일 이탈리아 볼...
